<a href="https://colab.research.google.com/github/An1ndya/IIT-Kharagpur/blob/main/Copy_of_COCO_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download and process COCO dataset**

In [1]:
from PIL import Image
import json
import os

#Removes old data files
cleanBeforeProcessing = True
#Image dimensions
imgSize = 256

#Download image dataset
!wget http://images.cocodataset.org/zips/train2017.zip
#Extract archive 
!unzip train2017.zip
#Clean up
!rm train2017.zip

#Resize all images
for root, dirs, files in os.walk("train2017"):
  for fname in files:
    if ".png" in fname or ".jpg" in fname:
      path = os.path.join(root, fname)
      im = Image.open(path)
      im_resized = im.resize((imgSize, imgSize))
      im_resized.save(path)
      print("Resizing: " + fname)

--2022-11-14 05:00:10--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.36.20
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.36.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip         1%[                    ] 190.58M  12.7MB/s    eta 26m 41s^C
Archive:  train2017.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of train2017.zip or
        train2017.zip.zip, and cannot find train2017.zip.ZIP, period.


In [ ]:
cleanBeforeProcessing = True
#Download captions
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
#Extract archive 
!unzip annotations_trainval2017.zip


--2022-11-11 20:20:46--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.169.73
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.169.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip.2’

annotations_trainva 100%[===================>] 241.19M  33.7MB/s    in 7.8s    

2022-11-11 20:20:54 (31.0 MB/s) - ‘annotations_trainval2017.zip.2’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import json
import os

if cleanBeforeProcessing:
  os.system('rm od-labelsonly.txt')
  os.system('rm od-labelsimgid.txt')

#Read images information
with open('annotations/instances_train2017.json', 'r') as f:
    array = json.load(f)

#Read all images paths
imagePaths = []
for root, dirs, files in os.walk("train2017"):
  for fname in files:
    if ".png" in fname or ".jpg" in fname:
      path = os.path.join(root, fname)
      imagePaths.append(path)


In [ ]:

#Read information for each image
idAlreadyProcessed = {}
count = 0
for info in array['annotations']:

  image_id = int(info['id'])
  temp = "000000000000"
  image_id_str = temp[:len(temp)-len(str(image_id))] + str(image_id)

  path = "train2017/" + image_id_str + ".jpg"
  label = info['category_id']

  #Choose first caption and image_id
  with open("od-labelsonly.txt", 'a') as out:
    out.write(str(label) + '\n')
  #Write this as CSV ... it will be easier to access later on
  with open("od-labelsimgid.txt", 'a') as out:
    out.write(str(image_id) + " : " + str(label) + '\n')

  count += 1
  if count%5000 == 0:
    print("Processed " + str(count))

Processed 5000
Processed 10000
Processed 15000
Processed 20000
Processed 25000
Processed 30000
Processed 35000
Processed 40000
Processed 45000
Processed 50000
Processed 55000
Processed 60000
Processed 65000
Processed 70000
Processed 75000
Processed 80000
Processed 85000
Processed 90000
Processed 95000
Processed 100000
Processed 105000
Processed 110000
Processed 115000
Processed 120000
Processed 125000
Processed 130000
Processed 135000
Processed 140000
Processed 145000
Processed 150000
Processed 155000
Processed 160000
Processed 165000
Processed 170000
Processed 175000
Processed 180000
Processed 185000
Processed 190000
Processed 195000
Processed 200000
Processed 205000
Processed 210000
Processed 215000
Processed 220000
Processed 225000
Processed 230000
Processed 235000
Processed 240000
Processed 245000
Processed 250000
Processed 255000
Processed 260000
Processed 265000
Processed 270000
Processed 275000
Processed 280000
Processed 285000
Processed 290000
Processed 295000
Processed 300000


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils 


class COCODataset(Dataset):
    """COCO dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.label = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.label.iloc[idx, 0])
        image = io.imread(img_name)
        label = self.label.iloc[idx, 1:]
        label = np.array([label])
        label = label.astype('int32').reshape(-1, 2)
        sample = {'image': image, 'label': label}

        # if self.transform:
        #     sample = self.transform(sample)

        return sample

In [ ]:
dataset = COCODataset(csv_file='od-labelsimgid.txt',
                                    root_dir='data/') # replace the locations

fig = plt.figure()

for i in range(len(dataset)):
    sample = dataset[i]

    print(i, sample['image'].shape, sample['label'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    plt.imshow(sample['image'])

    if i == 3:
        plt.show()
        break